# Model Training & Testing

In [8]:
import dcor
import numpy as np
import pandas as pd
import xgboost as xgb
import matplotlib.pyplot as plt
from sklearn.metrics import mean_squared_error, mean_absolute_error
from bayes_opt import BayesianOptimization


# --- Set global Matplotlib parameters ---
plt.rcParams["font.family"] = "serif"
plt.rcParams["figure.dpi"] = 1000
plt.rcParams["font.size"] = 20
plt.rcParams["axes.labelsize"] = 20
plt.rcParams["axes.titlesize"] = 18
plt.rcParams["legend.fontsize"] = 20
plt.rcParams["xtick.direction"] = "in"
plt.rcParams["ytick.direction"] = "in"
plt.rcParams["xtick.major.size"] = 5.0
plt.rcParams["xtick.minor.size"] = 3.0
plt.rcParams["ytick.major.size"] = 5.0
plt.rcParams["ytick.minor.size"] = 3.0
plt.rcParams["axes.linewidth"] = 1.5
plt.rcParams["legend.handlelength"] = 2.0

## Pseudo-LOOCV for Model Training

In [9]:
# --- 1. Load Data and Define Features ---
df = pd.read_csv('../datasets/features_final.csv', parse_dates=['Date'])
target = 'Load'
features_to_drop = [
    'Load', 'Date', 'Year', 'Month', 'Day', 'Day_Clipped',
    'DayOfYear', 'Timestep', 'Season', 'dow'
]
features = [col for col in df.columns if col not in features_to_drop]

# Prepare the data from Years 1 & 2 for the CV process
cv_df = df[df['Year'].isin([1, 2])].copy()
# Create a continuous month index for easy splitting
cv_df['ContinuousMonth'] = (cv_df['Year'] - 1) * 12 + cv_df['Month']

# --- 2. Define Helper Function for MAPE ---
def mean_absolute_percentage_error(y_true, y_pred):
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    # Avoid division by zero
    mask = y_true != 0
    return np.mean(np.abs((y_true[mask] - y_pred[mask]) / y_true[mask])) * 100

# --- 3. Set up and Run Cross-Validation ---
print("Starting time-series cross-validation...")

# Define the splits: (end_month_of_train_set)
split_points = [12, 15, 18, 21] # Corresponds to end of Year 1, Y2-Q1, Y2-Q2, Y2-Q3
results = []

for i, split_month in enumerate(split_points):
    fold_num = i + 1
    print(f"--- Running Fold {fold_num}/{len(split_points)} ---")
    
    # a. Split data for the current fold
    train_set = cv_df[cv_df['ContinuousMonth'] <= split_month]
    val_set = cv_df[cv_df['ContinuousMonth'] > split_month]
    
    X_train, y_train = train_set[features], train_set[target]
    X_val, y_val = val_set[features], val_set[target]
    
    # b. Initialize and train the model
    xgb_model = xgb.XGBRegressor(
        n_estimators=1000,
        learning_rate=0.05,
        objective='reg:squarederror',
        early_stopping_rounds=50,
        eval_metric='rmse',
        n_jobs=-1
    )
    xgb_model.fit(X_train, y_train, eval_set=[(X_val, y_val)], verbose=False)
    
    # c. Make predictions on the validation set
    predictions = xgb_model.predict(X_val)
    
    # d. Calculate metrics
    mse = mean_squared_error(y_val, predictions)
    rmse = np.sqrt(mse)
    mae = mean_absolute_error(y_val, predictions)
    mape = mean_absolute_percentage_error(y_val, predictions)
    energy_dist = dcor.energy_distance(predictions, y_val)
    
    results.append({
        'Fold': f"Fold {fold_num}",
        'Training Period': f"Year 1 -> Year 2, Month {split_month-12 if split_month > 12 else 12}",
        'Validation Period': f"Year 2, Month {split_month-11 if split_month > 12 else 1} -> 12",
        'MSE': mse,
        'RMSE': rmse,
        'MAE': mae,
        'MAPE (%)': mape,
        'Energy Distance': energy_dist
    })

# --- 4. Display CV Results ---
cv_results_df = pd.DataFrame(results)
display(cv_results_df.style.format({
    'MSE': '{:,.2f}', 'RMSE': '{:,.2f}', 'MAE': '{:,.2f}',
    'MAPE (%)': '{:.2f}', 'Energy Distance': '{:,.2f}'
}))

Starting time-series cross-validation...
--- Running Fold 1/4 ---
--- Running Fold 2/4 ---
--- Running Fold 3/4 ---
--- Running Fold 4/4 ---


,Fold,Training Period,Validation Period,MSE,RMSE,MAE,MAPE (%),Energy Distance
0,Fold 1,"Year 1 -> Year 2, Month 12","Year 2, Month 1 -> 12","33,272.85",182.41,127.96,5.93,2.20
1,Fold 2,"Year 1 -> Year 2, Month 3","Year 2, Month 4 -> 12","33,986.76",184.35,131.57,5.89,5.79
2,Fold 3,"Year 1 -> Year 2, Month 6","Year 2, Month 7 -> 12","40,644.16",201.60,144.84,6.12,8.99
3,Fold 4,"Year 1 -> Year 2, Month 9","Year 2, Month 10 -> 12","41,189.85",202.95,135.90,6.29,2.49


In [10]:
# --- 2. Split Data for Optimization and Final Test ---
# Years 1 & 2 will be used for the optimization process
# Year 3 is the final hold-out set for forecasting
opt_df = df[df['Year'].isin([1, 2])]
final_test_df = df[df['Year'] == 3]

# Further split the optimization data: Year 1 to train, Year 2 to validate
X_train_opt = opt_df[opt_df['Year'] == 1][features]
y_train_opt = opt_df[opt_df['Year'] == 1][target]
X_val_opt = opt_df[opt_df['Year'] == 2][features]
y_val_opt = opt_df[opt_df['Year'] == 2][target]

# --- 3. Define the Bayesian Optimization Function ---
# This function takes hyperparameters as input, trains a model, and returns a score to maximize.
def xgb_objective_function(max_depth, learning_rate, gamma, colsample_bytree, subsample):
    # The optimizer passes float values, but some parameters need to be integers
    params = {
        'max_depth': int(max_depth),
        'learning_rate': learning_rate,
        'gamma': gamma,
        'colsample_bytree': colsample_bytree,
        'subsample': subsample,
        'objective': 'reg:squarederror',
        'eval_metric': 'rmse',
        'n_jobs': -1,
        'tree_method': 'hist',
        'device': 'cuda'
    }
    
    model = xgb.XGBRegressor(n_estimators=1000, **params)
    
    # Use early stopping to find the best number of trees
    model.fit(X_train_opt, y_train_opt,
              eval_set=[(X_val_opt, y_val_opt)],
              verbose=False)
              
    predictions = model.predict(X_val_opt)
    rmse = np.sqrt(mean_squared_error(y_val_opt, predictions))
    
    # BayesianOptimization maximizes, so we return negative RMSE (lower is better)
    return -rmse

# --- 4. Define Hyperparameter Search Space and Run Optimization ---
param_bounds = {
    'max_depth': (3, 10),
    'learning_rate': (0.01, 0.3),
    'gamma': (0, 1),
    'colsample_bytree': (0.5, 1),
    'subsample': (0.5, 1)
}

optimizer = BayesianOptimization(
    f=xgb_objective_function,
    pbounds=param_bounds,
    random_state=42,
    verbose=2 # Set to 2 to see the search progress
)

print("Starting Bayesian hyperparameter search...")
# init_points: number of random points to start with
# n_iter: number of intelligent exploration steps
optimizer.maximize(init_points=5, n_iter=15)
print("\nSearch complete.")

# --- 5. Train Final Model with Best Hyperparameters ---
# Get the best parameters found by the optimizer
best_params = optimizer.max['params']
# Ensure max_depth is an integer
best_params['max_depth'] = int(best_params['max_depth'])

print("\nBest hyperparameters found:", best_params)

# Define the final training set (all of Years 1 & 2)
X_train_final = opt_df[features]
y_train_final = opt_df[target]
X_test_final = final_test_df[features]

# Initialize and train the final model on all available data
final_model = xgb.XGBRegressor(
    n_estimators=1000, # We can use early stopping here as well if we have a validation set
    objective='reg:squarederror',
    eval_metric='rmse',
    n_jobs=-1,
)

print("\nTraining final model on all data (Years 1 & 2) with optimal parameters...")
final_model.fit(X_train_final, y_train_final, verbose=False)
print("Final model training complete.")

# --- 6. Generate and Visualize Final Forecast ---
final_forecast = final_model.predict(X_test_final)
forecast_df = final_test_df[['Date']].copy()
forecast_df['Predicted_Load'] = final_forecast

fig, ax = plt.subplots(figsize=(18, 7))
forecast_df.plot(x='Date', y='Predicted_Load', ax=ax, legend=None)
ax.set_title('Final Forecasted Load for Year 3 (Optimized Model)', fontsize=16)
ax.set_xlabel('Date', fontsize=12)
ax.set_ylabel('Predicted Load', fontsize=12)
plt.grid(True)
plt.show()

# --- 7. Save Final Results ---
output_filename = 'final_forecast_optimized_year3.csv'
forecast_df.to_csv(output_filename, index=False)
print(f"\nFinal optimized forecast for Year 3 has been saved to '{output_filename}'.")

Starting Bayesian hyperparameter search...
|   iter    |  target   | max_depth | learni... |   gamma   | colsam... | subsample |
-------------------------------------------------------------------------------------


c:\Users\quang\anaconda3\Lib\site-packages\xgboost\core.py:729: UserWarning: [19:29:40] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\common\error_msg.cc:58: Falling back to prediction using DMatrix due to mismatched devices. This might lead to higher memory usage and slower performance. XGBoost is running on: cuda:0, while the input data is on: cpu.
Potential solutions:
- Use a data structure that matches the device ordinal in the booster.
- Set the device for booster before call to inplace_predict.

This warning will only be shown once.

  return func(**kwargs)


| 1         | -192.0133 | 5.6217808 | 0.2857071 | 0.7319939 | 0.7993292 | 0.5780093 |
| 2         | -180.8121 | 4.0919616 | 0.0268442 | 0.8661761 | 0.8005575 | 0.8540362 |
| 3         | -193.0705 | 3.1440914 | 0.2912738 | 0.8324426 | 0.6061695 | 0.5909124 |
| 4         | -183.9097 | 4.2838315 | 0.0982302 | 0.5247564 | 0.7159725 | 0.6456145 |
| 5         | -176.9484 | 7.2829702 | 0.0504532 | 0.2921446 | 0.6831809 | 0.7280349 |
| 6         | -177.0175 | 7.2866403 | 0.0508660 | 0.2961856 | 0.6657791 | 0.7258895 |
| 7         | -177.9482 | 7.9981210 | 0.01      | 0.0       | 0.9932771 | 0.8462049 |
| 8         | -189.4847 | 7.7992315 | 0.2767460 | 1.0       | 1.0       | 1.0       |
| 9         | -195.9906 | 7.5706202 | 0.3       | 0.0       | 0.7903411 | 0.5       |
| 10        | -176.2459 | 7.0915030 | 0.01      | 0.4253612 | 0.6718915 | 0.8889326 |
| 11        | -177.0315 | 7.0121938 | 0.01      | 0.5255595 | 0.7337187 | 0.5314469 |
| 12        | -176.6444 | 6.8271243 | 0.01      | 0.13


Final optimized forecast for Year 3 has been saved to 'final_forecast_optimized_year3.csv'.
